In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import scipy.stats
import pandas as pd
from sklearn.preprocessing import minmax_scale
import pymc3 as pm
import seaborn as sns

In [ ]:
mdf=pd.read_excel('MASTER.xlsx')

In [ ]:
rows=[]
root='NMR_data_deposition/'
for idx,mrow in mdf.iterrows():
    filename=mrow['File_name']
    with open(root+filename) as f:
        while True:
            line=f.readline()
            if "F1LEFT" in line:
                sline=line.split()
                F1start=sline[3]
                F1end=sline[-2]
            if "F2LEFT" in line:
                sline=line.split()
                F2start=sline[3]
                F2end=sline[-2]
            if "NROWS" in line:
                sline=line.split()
                nrows=sline[3]
            if "NCOLS" in line:
                sline=line.split()
                ncols=sline[3]
            if not line:
                break
        data=np.loadtxt(root+filename)
        row=pd.Series(data).to_dict()
        row['F1s']=F1start
        row['F1e']=F1end
        row['F2s']=F2start
        row['F2e']=F2end
        row['nrows']=nrows
        row['ncols']=ncols
        row['filename']=filename
        row['Peptide']=mrow['Peptide']
        row['Peptide conc. (mM)']=mrow['Peptide conc. (mM)']
        row['Survivin_concentration (mM)']=mrow['Survivin_concentration (mM)']
        rows.append(row)

df=pd.DataFrame(rows)
df

In [ ]:
%matplotlib
pepdict={}

peptides=["EZH2_172-211",
          "EED_128-152",
          "SUZ12_573-597",
          "JARID2_170-194",
         ]

nice_peptides=[r"$EZH2_{172-211}$",
          "$EED_{128-152}$",
          "$SUZ12_{573-597}$",
          "$JARID2_{170-194}$",
         ]
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
colors = colors+colors
fig, ax = plt.subplots(figsize=[6.4, 4.8],constrained_layout=True, dpi=600)
plots=[]
for cidx,peptide in enumerate(peptides):
    ms=[]
    logdatas=[]
    img=[]
    datas=[]
    cs=[]
    scs=[]
    
    if peptide in ["EZH2_172-211","SUZ12_573-597","EED_128-152","JARID2_170-194"]:
        row=df[df.filename=="190611_surv.txt"].iloc[0]
        data=np.array(row.iloc[0:524288],dtype='float64')
        datas.append(data)
        img.append(data.reshape(256,2048))
        cs.append(row["Peptide conc. (mM)"])
        scs.append(row['Survivin_concentration (mM)'])

    for idx,row in df[df.Peptide==peptide].iterrows():
        data=np.array(row.iloc[0:524288],dtype='float64')
        datas.append(data)
        img.append(data.reshape(256,2048))
        nrows=row['nrows']
        ncols=row['nrows']
        f1s=row['F1s']
        f1e=row['F1e']
        f2s=row['F2s']
        f2e=row['F2e']
        cs.append(row["Peptide conc. (mM)"])
        scs.append(row['Survivin_concentration (mM)'])

    for idx,data in enumerate(datas):
        data_noncorr=data[data>0]
        data=data_noncorr*scs[0]/scs[idx]
        logdata=data

        logdatas.append(logdata)
        sortdata=np.sort(logdata)[-200:]
        print(peptide,sortdata)
        m=np.mean(sortdata)
        ms.append(m)
    if peptide in ["EZH2_172-211","EED_128-152"]:
        minmax=minmax_scale(ms)
        fcs=cs[1:]
        fm=minmax[1:]
    else:
        minmax=minmax_scale(ms[0:-1])
        fcs=cs[1:-1]
        fm=minmax[1:]
    plot,=ax.semilogx(fcs,fm,label=peptide,marker="s",c=colors[cidx], )
    plt.xlim(0.0002,5)

    pepdict[peptide]=(fcs,fm)

    plots.append(plot)
ax.legend(plots,nice_peptides)
ax.set_ylabel(r'$(<I_{c}>_i - <I_{c}>_{min}) / (<I_{c}>_{max} - <I_{c}>_{min})$')
ax.set_xlabel(r'$Peptide\ concentration\ (mM)$')
plt.savefig('NMR_int.png',dpi=600)

In [ ]:
traces={}
ppcs={}
peptides=["EZH2_172-211",
          "EED_128-152",
          "SUZ12_573-597",
          "JARID2_170-194"
         ]

for cidx,peptide in enumerate(peptides):

    cs,ms=pepdict[peptide]
    with pm.Model() as model:
        c0 = pm.Uniform('c0', -10, 1)
        L = pm.Uniform('L', -3, 3)
        k = pm.Uniform('k', -10,10)
        shift = pm.Uniform('shift', -10,10)
        totalamp=pm.Deterministic('totalamp', shift+L/(1.0+pm.math.exp(-1.0*k*(np.log10(cs)-c0))))
        sigma = pm.Uniform('sigma', 0, 0.05)
        pred = pm.Normal('pred', mu=totalamp, sd=sigma, observed=np.array(ms))

    with model:

        trace = pm.sampling.sample(tune=20000, target_accept=0.95)
    with model:

        ppc = pm.sample_posterior_predictive(trace, samples=1000 ,model=model)
    traces[peptide]=trace
    ppcs[peptide]=ppc

In [ ]:
%matplotlib qt
rrows=[]
fig, ax = plt.subplots(figsize=[6.4, 2.4],constrained_layout=True, dpi=600)
for cidx,peptide in enumerate(peptides):
#pm.traceplot(trace,['c0'],combined=True)
    ax.set(xscale="log")
    ax = sns.kdeplot(10**traces[peptide]['c0'], label=peptide)
#    ax = sns.kdeplot(traces[peptide]['c0'], label=peptide)
#    ax.legend(nice_peptides)
    ax.set_xlabel(r'$Critical\ peptide\ concentration\ (mM)$')
    row={}
    row['Peptide']=peptide
    row['<L>']='%4.2f' % np.mean(traces[peptide]['L'])
    row['L HDI95%']='(%4.2f,%4.2f)' % tuple(pm.stats.hpd(traces[peptide]['L']))
    row['<k>']='%4.2f' % np.mean(traces[peptide]['k'])
    row['k HDI95%']='(%4.2f,%4.2f)' % tuple(pm.stats.hpd(traces[peptide]['k']))
    row['<c0>']='%.1E' % np.mean(10**traces[peptide]['c0'])
    row['c0 HDI95%']='(%.1E,%.1E)' % tuple(pm.stats.hpd(10**traces[peptide]['c0']))
    row['<sig>']='%4.2f' % np.mean(traces[peptide]['sigma'])
    row['sig HDI95%']='(%4.2f,%4.2f)' % tuple(pm.stats.hpd(traces[peptide]['sigma']))
    row['<shift>']='%4.2f' % np.mean(traces[peptide]['shift'])
    row['shift HDI95%']='(%4.2f,%4.2f)' % tuple(pm.stats.hpd(traces[peptide]['shift']))
    rrows.append(row)
    print (peptide,'L',np.mean(traces[peptide]['L']),pm.stats.hpd(traces[peptide]['L']))
    print (peptide,'k',np.mean(traces[peptide]['k']),pm.stats.hpd(traces[peptide]['k']))
    print (peptide,'c0',np.mean(10**traces[peptide]['c0']),pm.stats.hpd(10**traces[peptide]['c0']))
    print (peptide,'sigma',np.mean(traces[peptide]['sigma']),pm.stats.hpd(traces[peptide]['sigma']))
    print (peptide,'shift',np.mean(traces[peptide]['shift']),pm.stats.hpd(traces[peptide]['shift']))
#    plt.xlim(np.log10(0.0002),np.log10(5))
    
    plt.xlim(0.0002,5)
rdf=pd.DataFrame(rrows)
rdf.to_excel('NMR_table.xlsx')
plt.savefig('NMR_kde.png')

In [ ]:
%matplotlib
def logist (cs,shift,L,k,c0):
    return shift+L/(1.0+np.exp(-1.0*k*(np.log10(cs)-c0)))
fig,ax=plt.subplots()
pep='EZH2_172-211'
cts=np.logspace(-4,1,100)
cs,ms=pepdict[pep]
ax.semilogx(cs,ms,c='r', marker='o',ls='')
for step in traces[pep][::10]:
    ax.semilogx(cts, logist(cts,step['shift'],step['L'],step['k'],step['c0']), c='gray',alpha=0.1)

ax.set_ylabel(r'$(<I_{c}>_i - <I_{c}>_{min}) / (<I_{c}>_{max} - <I_{c}>_{min})$')
ax.set_xlabel(r'$Peptide\ concentration\ (mM)$')
plt.savefig('EZH2_172-211.svg')